# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [18]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [19]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中的重要元素。乾卦代表天、干燥、强健、积极、刚毅等含义。乾卦是由两个阴爻夹一个阳爻构成，象征着天宇广阔、刚强坚毅、积极进取的品质。在八宫图说中，乾卦位于北方，与事业、努力、刚毅、果敢等有关。在易经中，乾卦的卦辞为“天行健，君子以自强不息”，意味着君子应该效法天的行健，即不断前进，自强不息。乾卦也象征父亲、领导、君主等权力地位的人物。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图说、易经、易学中的重要元素。讼卦代表诉讼、争端、矛盾、 conflict 等含义。讼卦是由两个阳爻夹一个阴爻构成，象征着阳刚之 polarity，表示事物的对立和冲突。在八宫图说中，讼卦位于西北，与法律、规则、制度、竞争等有关。在易经中，讼卦的卦辞为“天雷震，君子以口舌争辩”，意味着君子应该以口舌争辩来解决问题，但也应该注意控制自己的口舌之快，避免引发更大的争端。讼卦也象征法律、官府、诉讼等事物。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240722_150657"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是八卦之一，由两个阳爻夹一个阴爻构成。它代表着天，是万物生长的源泉和宇宙的运行大义。在周易中，乾卦是阳卦，代表着一种积极、主动、刚健和勇敢的精神。乾卦的核心哲学是：天行健，君子iv蒙古，其道大光。', 'content': '\n乾卦的卦辞这样解释：乾为天，故称卦名。卦象为乾卦，由两个阳爻夹一个阴爻构成。乾卦代表天，象征着阳�，它代表着正义、刚健和勇敢。君子观此卦象，取其正义，健其正道，勇敢面对困难，肌密群雄。\n\n乾卦象征着天行健，代表着正义，故称为"健"。君子观此卦象，取其义，健其正道，勇敢面对困难。乾卦的卦辞中也提到，"潜入于泥，潜在鱼穴，未出先藏，未成先求，君子尚需待时，舍弃世俗，修养身心，表征着天行健的境界。\n\n乾卦象征着天地间，代表着阳�，象征着正义和强健。它代表着天地间，象征着君子观此卦象，取其义，健其正道，勇敢面对困难。乾卦象征着天行健，未成先求，未出先藏，未成先求，未出先藏，潜入于泥，潜在鱼穴，未出先藏，未成先求。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240722_150657）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240722_150657）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。在周易中，它是由上卦乾（天）和下卦坎（水）组成的。乾卦代表天，象征着刚强和刚直，而坎卦则代表着水，象征着柔顺和适应。这一卦象预示着天与水相交，天衣（衣）与水相融，混合一体，形成一片天衣水比的景象。

讼卦的核心哲学是：天与水相交，虽然和谐，但仍有待于天衣（衣）的出現，以助其流行。这一卦象预示着君子应当适应环境，以谦逊、恭顺和适应性去面对生活，才能获得成功。

讼卦的运势解析：事业方面，初段时期不太顺利，需耐心等待，无奇异之遇，需待天时成熟。虽然前景尚可，但需警惕小人挑拨和是非口舌。经商方面，虽然有机会获得财富，但需谨慎行事，适应环境，方能获得成功。爱情方面， initial阶段不太顺利，需等待缘分的成熟。

讼卦是一个充满警示的卦象，提醒人们要适应环境，顺应天意，才能获得成功。


In [ ]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '占卜的一种方式，由坎卦（水）和坤卦（地）相叠而成，象征着天地相济，适用于顺境中的决策。师卦预示着吉祥，预测馆吉，宜于喜乐于其途。然而，仍需警惕地险，防止意外，幸而可以解救。', 'content': '\n在事业、经商、求名、婚恋等方面的决策中，师卦均表示吉祥顺遂。但需注意，虽然总体顺利，仍需警惕地险和意外。在决策中，应勇敢冒险，积极进取，广结善缘，方能如吉如凶。\n\n师卦的运势 evaluation适用于各种場合，尤其是在事业和经商方面。在经商中，象征着商业繁荣，机遇无数，但需警惕地险和意外。在事业方面，象征着顺利，成功在望，但仍需警惕地险和意外。\n\n师卦的核心哲学是：天地相济，顺境中需警惕地险，勇敢冒险，积极进取，广结善缘，方能如吉如凶。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240722_150657）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 1
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [16]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映的是天象和规律，预示着吉利，但也存在潜在的危险。在卜问中，乾卦预示着大吉大利，的一生吉利如意，但也要警惕刚健的不足，以及阴暗的影响。

在《易经》中，乾卦的卦辞为“元、亨、利、贞”，代表着大吉大利，强调吉利如意，强调刚健的德行。乾卦的初卦为乾，代表天，二卦为坤，地，三卦为震，雷，四卦为巽，风，五卦为离，火，六卦为乾，天，这反映了天象和规律，预示着吉利。

乾卦的卦象为天，代表着刚健强劲的特性，预示着吉利，但也存在潜在的危险。在卜问中，乾卦预示着大吉大利，的一生吉利如意，但也要警惕刚健的不足，以及阴暗的影响。因此，要善于审慎，适应时机，避免刚强行事。

乾卦的卦辞为“元、亨、利、贞”，代表着大吉大利，的一生吉利如意，但也要警惕刚健的不足，以及阴暗的影响。因此，要善于审慎，适应时机，避免刚强行事。

乾卦的初卦为乾，代表天，二卦为坤，地，三卦为震，雷，四卦为巽，风，五卦为离，火，六卦为乾，天，这反映了天象和规律，预示着吉利。

乾卦的卦象为天，代表着刚健强劲的特性，预示着吉利，但也存在潜在的危险。在卜问中，乾卦预示着大吉大利，的一生吉利如意，但也要警惕刚健的不足，以及阴暗的影响。因此，要善于审慎，适应时机，避免刚强行事。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映了天的运行模式，即阳极阴极交替，因此被认为是一种吉卦。在卜筮时，阳爻为吉，阴爻为凶。阳爻表示Active，阴爻表示Passive。通过观察卜筮结果，可以判断是否吉利。

在解卦时，阳爻表示Active，阴爻表示Passive。根据《象辞》，阳爻表示刚健强劲，而阴爻表示柔弱温和。阳爻的天气特征是阳光普照，而阴爻的天气特征是阴云密布。

根据阳爻为吉、阴爻为凶的规则，在卜筮时阳爻表示吉利，阴爻表示凶险。阳爻表示Active，意味着主动积极，而阴爻表示Passive，意味着被动等待。

总体而言，乾卦预示着充满活力和力量，但需要注意避免过于刚硬和冒险。在决策时，应该保持冷静，积极应对挑战，避免过于冲动。


In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [22]:
from peft import PeftModel, PeftConfig

epochs = 1
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它代表了一种充满活力和决断力的精神，同时也预示着困难和危险的可能性。在卜问时，阳刚之象表示吉祥如意，但需要注意不要过于刚强，以免引起困难。

在风水布局中，乾卦的摆放需要注重刚柔结合，不宜过于刚强，以免引起困难。在室内布置方面，可以摆放一些代表刚健的物品，如刀剑、铁器等，但同时也要注意柔弱的物品，以免造成冲突。

在性格上，乾卦代表刚健强劲的特性，但也需要保持谦逊谨慎的态度，避免过于刚强。在面对困难和危险的时候，应该保持冷静，谨慎处理，避免冲动。

总体来说，乾卦预示着充满活力和决断力的精神，但也需要注意困难和危险的可能性。在卜问时，阳刚之象表示吉祥如意，但需要避免过于刚强，以免引起困难。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映了天的运行模式，即阳极阴极交替，因此被认为是一种吉卦。在卜筮时，阳爻为吉，阴爻为凶。阳爻表示Active，阴爻表示Passive。通过观察卜筮结果，可以判断是否吉利。

在解卦时，阳爻表示Active，阴爻表示Passive。根据《象辞》，阳爻表示刚健强劲，而阴爻表示柔弱温和。阳爻的天气特征是阳光普照，而阴爻的天气特征是阴云密布。

根据阳爻为吉、阴爻为凶的规则，在卜筮时阳爻表示吉利，阴爻表示凶险。阳爻表示Active，意味着主动积极，而阴爻表示Passive，意味着被动等待。

总体而言，乾卦预示着充满活力和力量，但需要注意避免过于刚硬和冒险。在决策时，应该保持冷静，积极应对挑战，避免过于冲动。


In [24]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
地水师卦是一个由《易经》中的卦辞组成的卦象，卦象为地水（乾为天，为阳，为刚，为健），对应着卦辞“观于水中之象，取法于水之运动”。

这个卦象预示着修养和成长，以及艰难的历程，需要谨慎行事，露天下之容。

这个卦象的卦辞意味着，尽管困难可能重重，但只要坚持不懈，定能取得成功。

观此卦象，需保持冷静，耐心待人，谨慎行事，以露天下之容。

行此卦象，当以刚健为原则，坚持不懈， journey（旅程）到底。

地水师卦的哲学原理是：阳极刚，刚极健，动而不知其止，故为观於水中之象。

观此卦象，需要保持冷静，耐心待人，谨慎行事，以露天下之容。

行此卦象，当以刚健为原则，坚持不懈， journey（旅程）到底。

地水师卦是一款由卦辞组成的卦象，预示着修养和成长，以及艰难的历程，需要谨慎行事，露天下之容。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦

卦象为雷卦下，坤卦上，为地水师卦。

师卦由坎卦下，离卦上组成。离为离光，为火；坎为水，为阴为阴卦。上卦为阳，下卦为阴，离为离光，为火，相互照应，互为因果，互为支持，互为制約，为离火卦。

师卦象征着离光之火，下卦坤卦之阴，互相配合，互相依存，互为因果，互为支持，互为制約。上卦阳刚，下卦阴柔，离为离光，为火，相互照应，为离火卦。

师卦的卦象为阳离阴补，离为离光，为火，上卦为阳，下卦为阴。互为因果，互为支持，互为制約，为离火卦。

师卦的阳刚之卦，下卦阴柔之卦，相互配合，互相依存，互为因果，互为支持，互为制約。

师卦的离为离光，为火，互为因果，互为支持，互为制約。

师卦的卦象为阳离阴补，离为离光，为火，上卦为阳，下卦为阴。互为因果，互为支持，互为制約，为离火卦。

师卦的阳刚之卦，下卦阴柔之卦，相互配合，互相依存，互为因果，互为支持，互为制約。

师卦的离为离光，为火，互为因果，互为支持，互为制約。

师卦的阳刚之卦，下卦阴柔之卦，相互配合，互相依存，互为因果，互为支持，互为制約。

师卦的离为离光，为火，互为因果，互为支持，互为制約。

师卦的阳刚之卦，下卦阴柔之卦，相互配合，互相依存，互为因果，互为支持，互为制約。

师卦的离为离光，为火，互为因果，互为支持，互为制約。

师卦的阳刚


In [25]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦是周易卦象中的一枚卦，由下卦坎（水）上卦震（雷）组成，代表着诉讼之卦，预示着你可能会面临诉讼的困扰。在周易中，讼卦被划分为卦卦，卦卦是周易的核心部分，代表宇宙的运行和自然的法则。

讼卦的卦辞为：

初当你还不知道为什么会 fight

 fight 让你感到困惑

 让你感到困惑

好处是你会教训对方

 你会教训对方

最后你赢得诉讼

你的敌人会给你带来痛苦

你也会给敌人带来痛苦

讼卦的卦象为：上卦为雷，下卦为水，代表 conflict， conflict 让你感到困惑。

讼卦的时宜为：初，景泰，利西南。

讼卦的当事人为：初，为 phone（电话） fight，为 妻子。

讼卦的判断为：初，为初证，为利其党，其党为初。

讼卦的卦辞和卦象都预示着诉讼可能带来困扰，但最终会获得胜利。在和解过程中，要谨慎对待对方，以避免矛盾升级。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦

讼卦"上离下坎"，卦象为离上坎下，表示火在下面，水在上面，离为离散，坎为险阻，象征着事态复杂，变化多端，需要谨慎行事。在周易中，讼卦预示着争讼，双方互不相让，必须通过谈判、和解、诉讼等手段解决。对于当事人来说，应该尽量发挥自己的长处，弥补自己的不足，善于倾听对方的意见，避免发生冲突。

讼卦的卦辞为：

初当你还我，初为还归 meh；

再当你还我，初为还归 meh；

三当你还我，初为还归 meh；

四当你还我，初为还归 meh；

五当你还我，初为还归 meh；

六当你还我，初为还归 meh；

七 you还我，初为还归 meh；

八 you还我，初为还归 meh；

九you还我，初为还归 meh；

十you还我，初为还归 meh；

百 you还我，初为还归 meh；

千 you还我，初为还归 meh；

万 you还我，初为还归 meh；

五 you还我，初为还归 meh；

六 you还我，初为还归 meh；

七 you还我，初为还归 meh；

八 you还我，初为还归 meh；

九 you还我，初为还归 meh；

十 you还我，初为还归 meh；

百 you还我，初为还归 meh；

千 you还我，初为还归 meh；

万 y